In [23]:
!pip install optuna xgboost scikit-learn pandas numpy --quiet


In [24]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import xgboost as xgb

train_df = pd.read_csv("/kaggle/input/ml-project-data/train.csv")
test_df = pd.read_csv("/kaggle/input/ml-project-data/test.csv")
print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")


Train shape: (15533, 18), Test shape: (5225, 17)


In [27]:
target_col = "WeightCategory"
print(f"target column: {target_col}")

target column: WeightCategory


In [29]:
X_full = train_df.drop(columns=[target_col, "id"], errors="ignore").copy()
X_test = test_df.drop(columns=["id"], errors="ignore").copy()

for col in X_full.columns:
    if X_full[col].dtype == 'object':
        X_full[col] = X_full[col].astype('category')
        if col in X_test.columns:
            X_test[col] = X_test[col].astype('category')
            all_cats = pd.Categorical(pd.concat([X_full[col].astype(str), X_test[col].astype(str)])).categories
            X_full[col] = pd.Categorical(X_full[col].astype(str), categories=all_cats)
            X_test[col] = pd.Categorical(X_test[col].astype(str), categories=all_cats)

In [30]:
if train_df[target_col].dtype == 'object':
    from sklearn.preprocessing import LabelEncoder
    target_le = LabelEncoder()
    y = target_le.fit_transform(train_df[target_col].astype(str))
else:
    target_le = None
    y = train_df[target_col].values

In [31]:
def has_cuda_gpu():
    try:
        import cupy as cp
        return cp.cuda.runtime.getDeviceCount() > 0
    except Exception:
        return False

xgb_device = "cuda" if has_cuda_gpu() else "cpu"
print(f"XGBoost device: {xgb_device}")

def version_ge(ver, base="2.0"):
    def to_tuple(s):
        parts = s.split(".")
        nums = []
        for p in parts:
            if p.isdigit():
                nums.append(int(p))
            else:
                num = "".join(ch for ch in p if ch.isdigit())
                if num:
                    nums.append(int(num))
                break
        while len(nums) < 2:
            nums.append(0)
        return tuple(nums[:2])
    return to_tuple(xgb.__version__) >= to_tuple(base)

SUPPORTS_DEVICE_PARAM = version_ge(xgb.__version__, "2.0")

def build_params_for_device(base_params, device):
    params = dict(base_params)
    if SUPPORTS_DEVICE_PARAM:
        params.update({"tree_method": "hist", "device": device})
    else:
        if device == "cuda":
            params.update({"tree_method": "gpu_hist", "predictor": "gpu_predictor"})
        else:
            params.update({"tree_method": "hist"})
    return params


XGBoost device: cpu


In [34]:
def objective(trial):
    base_params = {
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 500, 3000),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "gamma": trial.suggest_float("gamma", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.001, 50.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "max_bin": trial.suggest_int("max_bin", 256, 1024),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
        "objective": "multi:softprob",
        "num_class": int(len(np.unique(y))),
        "random_state": 42,
        "n_jobs": -1,
        "eval_metric": "mlogloss",
        "enable_categorical": True,
    }
    params = build_params_for_device(base_params, xgb_device)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for tr_idx, val_idx in skf.split(X_full, y):
        X_tr, X_val = X_full.iloc[tr_idx], X_full.iloc[val_idx]
        y_tr, y_val = y[tr_idx], y[val_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=100,
            verbose=False
        )
        probs = model.predict_proba(X_val)
        preds = probs.argmax(axis=1)
        scores.append(accuracy_score(y_val, preds))

    return float(np.mean(scores))


In [35]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("Best CV Accuracy:", study.best_value)
print("Best Params:", study.best_params)


[I 2025-10-26 10:32:28,207] A new study created in memory with name: no-name-4c5c7377-9877-439b-9148-10bb600b7791


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

[I 2025-10-26 10:33:13,009] Trial 0 finished with value: 0.9039466373316095 and parameters: {'max_depth': 6, 'learning_rate': 0.24517932047070642, 'n_estimators': 2330, 'min_child_weight': 12, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.49359671220172163, 'gamma': 0.5808361216819946, 'reg_lambda': 11.752647960576219, 'reg_alpha': 6.011150117432088, 'max_bin': 800, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.9039466373316095.


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

...

[I 2025-10-26 11:22:30,421] Trial 97 finished with value: 0.9072941870868411 and parameters: {'max_depth': 12, 'learning_rate': 0.058782248590823385, 'n_estimators': 501, 'min_child_weight': 17, 'subsample': 0.8625711367114359, 'colsample_bytree': 0.40102655291745115, 'gamma': 0.5245854006043574, 'reg_lambda': 0.004178620792048689, 'reg_alpha': 1.1780987173852013, 'max_bin': 382, 'grow_policy': 'depthwise'}. Best is trial 37 with value: 0.9086462220717151.


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

[I 2025-10-26 11:22:53,410] Trial 98 finished with value: 0.906328563277861 and parameters: {'max_depth': 11, 'learning_rate': 0.07615795579613419, 'n_estimators': 701, 'min_child_weight': 18, 'subsample': 0.8282759661345875, 'colsample_bytree': 0.41491796792498786, 'gamma': 0.5425614871081946, 'reg_lambda': 0.004075874533819046, 'reg_alpha': 1.3512554510771344, 'max_bin': 381, 'grow_policy': 'depthwise'}. Best is trial 37 with value: 0.9086462220717151.


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`se

[I 2025-10-26 11:23:08,793] Trial 99 finished with value: 0.8937104612458295 and parameters: {'max_depth': 6, 'learning_rate': 0.06013341778484783, 'n_estimators': 618, 'min_child_weight': 16, 'subsample': 0.8621265040275322, 'colsample_bytree': 0.4565254040755663, 'gamma': 8.495728753331672, 'reg_lambda': 0.006681920448092646, 'reg_alpha': 0.6297805255677655, 'max_bin': 360, 'grow_policy': 'depthwise'}. Best is trial 37 with value: 0.9086462220717151.
Best CV Accuracy: 0.9086462220717151
Best Params: {'max_depth': 10, 'learning_rate': 0.09841433416386688, 'n_estimators': 995, 'min_child_weight': 16, 'subsample': 0.8542578844625923, 'colsample_bytree': 0.43651370156855485, 'gamma': 0.7669387593265391, 'reg_lambda': 0.00801994413954166, 'reg_alpha': 0.7731462573746901, 'max_bin': 389, 'grow_policy': 'lossguide'}


In [36]:
best_params = study.best_params.copy()
best_params.update({
    "objective": "multi:softprob",
    "num_class": int(len(np.unique(y))),
    "random_state": 42,
    "n_jobs": -1,
    "eval_metric": "mlogloss",
    "enable_categorical": True,
})
best_params = build_params_for_device(best_params, xgb_device)

final_model = xgb.XGBClassifier(**best_params)
final_model.fit(X_full, y)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.43651370156855485, device='cpu',
              early_stopping_rounds=None, enable_categorical=True,
              eval_metric='mlogloss', feature_types=None,
              gamma=0.7669387593265391, grow_policy='lossguide',
              importance_type=None, interaction_constraints=None,
              learning_rate=0.09841433416386688, max_bin=389,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=16, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=995, n_jobs=-1, num_class=7,
              num_parallel_tree=None, ...)

In [37]:
probs_test = final_model.predict_proba(X_test)
test_preds = probs_test.argmax(axis=1)

if target_le is not None:
    test_preds = target_le.inverse_transform(test_preds)

submission_id = test_df["id"] if "id" in test_df.columns else np.arange(len(test_df))
submission = pd.DataFrame({"id": submission_id, target_col: test_preds})
submission.to_csv("submission.csv", index=False)
print("submission.csv created successfully!")
submission.head()


submission.csv created successfully!


,id,WeightCategory
0,15533,Obesity_Type_III
1,15534,Overweight_Level_I
2,15535,Overweight_Level_II
3,15536,Obesity_Type_II
4,15537,Normal_Weight
